# Create HINO Vectors
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Create vectors for Chems/Diseases on the Human Interaction Network Ontology. I'll get the Chem/Dis - pathways associations from CTD. These are inferred (I believe from the gene associations...?)

In [107]:
import pandas as pd
import numpy as np
import scipy as sp
import subprocess
import math
import re

### First we need to get the HINO IDs from the REACT and KEGG IDs that CTD uses

In [108]:
# Import HINO CSV
# Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,alternative term,bearer of,before,before or simultaneous with,BFO CLIF specification label,BFO OWL specification label,causal relation between processes,causally downstream of,causally downstream of or within,causally related to,causally upstream of,causally upstream of or within,"causally upstream of, negative effect","causally upstream of, positive effect",Contributor,controls,Coverage,Creator,curator note,database_cross_reference,Date,definition,definition source,depends on,Description,directly activates,directly inhibits,directly negatively regulates,directly positively regulates,directly provides input for,directly regulates,during which ends,during which starts,editor note,editor preferred term,elucidation,encompasses,ends,ends after,ends during,ends with,example of literature mining keyword usage,example of usage,expand expression to,Format,functionally related to,happens during,has activator,has associated axiom(fol),has associated axiom(nl),has component,has component activity,has component process,has curation status,has direct input,has effector activity,has input,has interaction type,has keyword dependency pattern,has literature mining keywords,has output,has part,has participant,has role,has_alternative_id,has_broad_synonym,has_controller,has_exact_synonym,has_narrow_synonym,has_next_step,has_obo_namespace,has_rank,has_related_synonym,http://data.bioontology.org/metadata/obo/part_of,http://data.bioontology.org/metadata/prefixIRI,http://data.bioontology.org/metadata/treeView,http://purl.obolibrary.org/obo/hasExactSynonym,http://purl.obolibrary.org/obo/hasNarrowSynonym,http://purl.obolibrary.org/obo/hasOBONamespace,http://purl.obolibrary.org/obo/id,http://purl.obolibrary.org/obo/mi#PSI-MI_slim,http://www.biopax.org/release/biopax-level3.owl#controlled,http://www.biopax.org/release/biopax-level3.owl#controlType,http://www.biopax.org/release/biopax-level3.owl#name,http://www.biopax.org/release/biopax-level3.owl#pathwayOrder,http://www.biopax.org/release/biopax-level3.owl#product,http://www.geneontology.org/formats/oboInOwl#created_by,http://www.geneontology.org/formats/oboInOwl#creation_date,http://www.geneontology.org/formats/oboInOwl#id,http://www.obofoundry.org/ro/ro.owl#has_part,http://www.obofoundry.org/ro/ro.owl#located_in,http://www.w3.org/2000/01/rdf-schema#comment,http://www.w3.org/2000/01/rdf-schema#isDefinedBy,http://www.w3.org/2004/02/skos/core#notation,http://xmlns.com/foaf/0.1/homepage,http://xmlns.com/foaf/0.1/page,immediately causally downstream of,immediately causally upstream of,immediately preceded by,immediately precedes,imported from,in_subset,indirectly activates,indirectly inhibits,inheres in,inheres in part of,input of,is direct form of,is indirect form of,label,lacks part,Language,logical macro assertion,logical macro assertion on a property,logical macro assertion on an annotation property,logical macro assertion on an object property,mereotopologically related to,negatively regulated by,negatively regulates,OBO foundry unique label,obsolete has direct output,obsolete preceded by,output of,overlaps,part of,participates in,positively regulated by,positively regulates,preceded by,precedes,Publisher,realized in,realizes,regulated by,regulates,related via dependence to,Relation,Resource Identifier,Resource Type,Rights Management,simultaneous with,Source,starts,starts before,starts during,starts with,Subject and Keywords,temporal interpretation,temporally related to,term editor,Title,transitively provides input for
hino = pd.read_csv('/home/ian/Downloads/HINO.csv')#.iloc[:,0:4]#, usecols=['uri', 'label', 'synonyms', 'defs'])

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,12,13,18,25,41,42,43,48,49,50,56,57,61,65,66,70,71,73,76,79,81,83,84,85,86,87,94,95,100,126,135,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
hino.sample(4)

,Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,alternative term,bearer of,...,starts,starts before,starts during,starts with,Subject and Keywords,temporal interpretation,temporally related to,term editor,Title,transitively provides input for
7614,http://purl.obolibrary.org/obo/UniProt_P26718,UniProt_P26718,NaN,NaN,False,NaN,NaN,http://purl.obolibrary.org/obo/PR_000000001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6753,http://purl.obolibrary.org/obo/HINO_0017422,Coatomer:Arf1-GTP:GAP lattice formation on gol...,NaN,"Reviewed: Rush, MG, 2008-01-11 00:00:00|Edited...",False,NaN,NaN,http://purl.obolibrary.org/obo/INO_0000040,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10891,http://purl.obolibrary.org/obo/UniProt_P33032,UniProt_P33032,NaN,NaN,False,NaN,NaN,http://purl.obolibrary.org/obo/PR_000000001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25024,http://purl.obolibrary.org/obo/CHEBI_59640,N-acetylglucosamine,N-acetylglucosamines,An N-acylglucosamine where the N-acyl group is...,False,NaN,NaN,http://purl.obolibrary.org/obo/CHEBI_21638|htt...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# examine the many cols
hino.iloc[7, 70:130]

has participant                                                                                               NaN
has role                                                                                                      NaN
has_alternative_id                                                                                            NaN
has_broad_synonym                                                                                             NaN
has_controller                                                                                                NaN
has_exact_synonym                                                                                             NaN
has_narrow_synonym                                                                                            NaN
has_next_step                                                                                                 NaN
has_obo_namespace                                                                       

In [111]:
hino.loc[6:8, ['http://www.obofoundry.org/ro/ro.owl#has_part','label', 'Definitions', 
                'has_exact_synonym', 'has_related_synonym']]

,http://www.obofoundry.org/ro/ro.owl#has_part,label,Definitions,has_exact_synonym,has_related_synonym
6,NaN,NaN,NaN,NaN,NaN
7,http://purl.obolibrary.org/obo/UniProt_P17252|...,"Protein kinase C (alpha, gamma, delta)",Converted from EntitySet in Reactome|Reactome ...,NaN,NaN
8,http://purl.obolibrary.org/obo/HINO_0006569,PathwayStep1895,NaN,NaN,NaN


In [112]:
hino.rename(columns={'http://www.obofoundry.org/ro/ro.owl#has_part': 'hasPart'}, inplace=True)

In [113]:
# # Use regex to get the reactome ID from Definitions
# r = re.compile('Reactome DB_ID: [0-9]*')
# vmatch = np.vectorize(lambda x:bool(r.match(x)))

# # func to extract the reactome bit
# def get_react_id(x, r):
#     if x is not None:
#         results = r.search(x)
#         if results is None: return None
#         else: return results[0].replace('Reactome DB_ID: ', 'REACT:R-HSA-')
#     else:
#         return None

# # Convert to str and extract
# hino['Definitions'] = hino.Definitions.astype(str)
# hino['Definitions'] = hino.Definitions.map(lambda x: get_react_id(x,r))

# hino = hino.dropna(subset=['Definitions'])
# print(len(hino))

In [114]:
# Actually instead Use regex to get the Uniprot ID from Definitions
r = re.compile('UniProt_[^,| ]*')
vmatch = np.vectorize(lambda x:bool(r.match(x)))

# func to extract the reactome bit
def get_uniprot_id(x, r):
    if x is not None:
        results = r.findall(x)
        if results is None: return None
        else: return results
    else:
        return None

# Convert to str and extract
hino['hasPart'] = hino['hasPart'].astype(str)
hino['Uniprot'] = hino.hasPart.map(lambda x: get_uniprot_id(x,r))

In [115]:
hino = hino[['Class ID', 'Definitions', 'hasPart', 'Uniprot']]

In [116]:
# look at uniprot col
# [x for x in hino.Uniprot if x][:3]

In [117]:
hino.sample(6)

,Class ID,Definitions,hasPart,Uniprot
7861,http://purl.obolibrary.org/obo/HINO_0020540,"Authored: Jupe, S, 2009-03-16 10:02:22|G beta:...",nan,[]
31899,http://purl.obolibrary.org/obo/HINO_0026178,has a Stoichiometric coefficient of 2|Reactome...,http://purl.obolibrary.org/obo/HINO_0026184,[]
17069,http://purl.obolibrary.org/obo/UniProt_Q5HYK7,NaN,nan,[]
12639,http://purl.obolibrary.org/obo/UniProt_O43524,NaN,nan,[]
4310,http://purl.obolibrary.org/obo/UniProt_Q9BW92,NaN,nan,[]
2973,http://purl.obolibrary.org/obo/HINO_0009134,"At the beginning of this reaction, 1 molecule ...",nan,[]


In [118]:
# create map of uniprot ID to URI
#First we need to split uniprot lists into multiple cols RUN ONLY ONCE per
s = hino.Uniprot.apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'Uniprot'
hino = hino.join(s.apply(lambda x: pd.Series(x)))
hino = hino.reset_index(drop=True)
hino.rename({0:"UniprotID"}, axis='columns', inplace=True)
hino.drop('Uniprot', axis=1, inplace=True)

In [119]:
# Now that we have one uniprot id per row (max) let's make a map
uniprot_hino = dict(zip(hino.dropna(subset=['UniprotID']).UniprotID, hino.dropna(subset=['UniprotID'])['Class ID']))

In [120]:
# Here we export the dictionary in a way that's easily imported as dict
import pickle 

with open('Uniprot_HINO_map'+ '.pkl', 'wb') as f:
        pickle.dump(uniprot_hino, f, pickle.HIGHEST_PROTOCOL)

In [121]:
# C/D HINOuri
chem_g_uni = pd.read_csv('CHEM_GENE_UNIPROT.csv')
dis_g_uni = pd.read_csv('DIS_GENE_UNIPROT.csv')

In [122]:
dis_g_uni['UniprotID'] = dis_g_uni.UniprotID.map(lambda x: 'UniProt_' + x)
chem_g_uni['UniprotID'] = chem_g_uni.UniprotID.map(lambda x: 'UniProt_' + x)

In [123]:
dis_g_uni['Hino'] = dis_g_uni.UniprotID.map(lambda x: uniprot_hino.get(x))
chem_g_uni['Hino'] = chem_g_uni.UniprotID.map(lambda x: uniprot_hino.get(x))

In [124]:
chem_g_uni.head()

,GeneID,UniprotID,ChemicalID,Hino
0,1,UniProt_P04217,D005459,None
1,1,UniProt_V9HWD8,D005459,None
2,2,UniProt_P01023,D005620,http://purl.obolibrary.org/obo/HINO_0003920
3,131076,UniProt_C9JQ41,D005485,None
4,131076,UniProt_Q4VC31,D005485,None


In [101]:
dis_g_uni.head()

,GeneID,UniprotID,DiseaseID,Hino,ReactID,HINO2
0,1,UniProt_P04217,MESH:D012559,None,798695,None
1,1,UniProt_V9HWD8,MESH:D012559,None,None,None
2,2,UniProt_P01023,MESH:D009404,http://purl.obolibrary.org/obo/HINO_0003920,963896,None
3,131076,UniProt_C9JQ41,NaN,None,None,None
4,131076,UniProt_Q4VC31,NaN,None,None,None


In [125]:
## This was an attempt to get more HINO links via react IDs, didn't really work - might have but only added
## 6 unique HINO IDs.... though added plenty of non unique. Overall likely not worth the effort but check if can
## # Use regex to get the reactome ID from Definitions
# r = re.compile('Reactome DB_ID: [0-9]*')
# vmatch = np.vectorize(lambda x:bool(r.match(x)))

# # func to extract the reactome bit
# def get_react_id(x, r):
#     if x is not None:
#         results = r.search(x)
#         if results is None: return None
#         else: return results[0].replace('Reactome DB_ID: ', '')
#     else:
#         return None

# Convert to str and extract
# hino['Definitions'] = hino.Definitions.astype(str)
# hino['Definitions'] = hino.Definitions.map(lambda x: get_react_id(x,r))

# hino = hino.dropna(subset=['Definitions'])
# print(len(hino))

# # make the map
# react_map = dict(zip(hino.Definitions, hino['Class ID']))

# # Now we have a react-hino map we need a uniprot-react map
# reactome = pd.read_csv('../UniProt2Reactome.txt', delimiter="\t", header=None) #error_bad_lines=False)
# reactome.rename(columns={1:'ReactID', 0:'UniprotID'}, inplace=True)
# reactome['UniprotID'] = reactome.UniprotID.map(lambda x: 'UniProt_' + x)
# reactome['ReactID'] = reactome.ReactID.map(lambda x: x[7:])
# uniprot_react = dict(zip(reactome['UniprotID'], reactome['ReactID']))  

# # Add the react IDs to the Chems/Genes (based on their associated uniprots)
# chem_g_uni['ReactID'] = chem_g_uni.UniprotID.map(lambda x: uniprot_react.get(x))
# dis_g_uni['ReactID'] = dis_g_uni.UniprotID.map(lambda x: uniprot_react.get(x))

# # Now create a second HINO column from the react IDs
# chem_g_uni['HINO2'] = chem_g_uni.ReactID.map(lambda x: react_map.get(x))
# dis_g_uni['HINO2'] = dis_g_uni.ReactID.map(lambda x: react_map.get(x))

In [106]:
chem_g_uni[chem_g_uni.HINO2.notna()]

,GeneID,UniprotID,ChemicalID,Hino,ReactID,HINO2
1001,629,UniProt_P00751,D005557,http://purl.obolibrary.org/obo/HINO_0006409,77606,http://purl.obolibrary.org/obo/HINO_0002640
1138,710,UniProt_P05155,D004997,http://purl.obolibrary.org/obo/HINO_0005722,77606,http://purl.obolibrary.org/obo/HINO_0002640
1140,712,UniProt_P02745,D004997,http://purl.obolibrary.org/obo/HINO_0006441,77606,http://purl.obolibrary.org/obo/HINO_0002640
1142,713,UniProt_P02746,D005492,http://purl.obolibrary.org/obo/HINO_0006441,77606,http://purl.obolibrary.org/obo/HINO_0002640
1144,714,UniProt_P02747,D004997,http://purl.obolibrary.org/obo/HINO_0006441,77606,http://purl.obolibrary.org/obo/HINO_0002640
1145,715,UniProt_P00736,D004221,http://purl.obolibrary.org/obo/HINO_0006475,77606,http://purl.obolibrary.org/obo/HINO_0002640
1147,716,UniProt_P09871,D005557,http://purl.obolibrary.org/obo/HINO_0006380,77606,http://purl.obolibrary.org/obo/HINO_0002640
1151,717,UniProt_P06681,C069837,http://purl.obolibrary.org/obo/HINO_0006403,77606,http://purl.obolibrary.org/obo/HINO_0002640
1156,718,UniProt_P01024,C070081,http://purl.obolibrary.org/obo/HINO_0003988,77606,http://purl.obolibrary.org/obo/HINO_0002640
1159,719,UniProt_Q16581,D005492,http://purl.obolibrary.org/obo/HINO_0020358,77606,http://purl.obolibrary.org/obo/HINO_0002640


In [55]:
temp = list(react_map.keys())

In [72]:
temp2 = dis_g_uni.ReactID.copy()
temp2 = temp2.dropna()

In [73]:
sorted(temp2)

['REACT:R-BTA-211945',
 'REACT:R-BTA-389948',
 'REACT:R-BTA-389948',
 'REACT:R-BTA-6798695',
 'REACT:R-BTA-9027307',
 'REACT:R-CEL-110320',
 'REACT:R-CEL-110320',
 'REACT:R-CEL-110357',
 'REACT:R-CEL-110362',
 'REACT:R-CEL-110362',
 'REACT:R-CEL-112310',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-112314',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114508',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-114608',
 'REACT:R-CEL-1169408',
 'REACT:R-CEL-1169408',
 'REACT

In [74]:
sorted(temp)[:-1]

['REACT:R-HSA-1006146',
 'REACT:R-HSA-1006173',
 'REACT:R-HSA-1008206',
 'REACT:R-HSA-1008217',
 'REACT:R-HSA-1008228',
 'REACT:R-HSA-1008229',
 'REACT:R-HSA-1008234',
 'REACT:R-HSA-1008247',
 'REACT:R-HSA-1008252',
 'REACT:R-HSA-1011577',
 'REACT:R-HSA-1011595',
 'REACT:R-HSA-1011605',
 'REACT:R-HSA-1012969',
 'REACT:R-HSA-1012978',
 'REACT:R-HSA-1012988',
 'REACT:R-HSA-1013010',
 'REACT:R-HSA-1013011',
 'REACT:R-HSA-1013017',
 'REACT:R-HSA-1013018',
 'REACT:R-HSA-1013019',
 'REACT:R-HSA-1013836',
 'REACT:R-HSA-1013858',
 'REACT:R-HSA-1013880',
 'REACT:R-HSA-1014260',
 'REACT:R-HSA-1015692',
 'REACT:R-HSA-1015693',
 'REACT:R-HSA-1015695',
 'REACT:R-HSA-1015696',
 'REACT:R-HSA-1015697',
 'REACT:R-HSA-1015817',
 'REACT:R-HSA-1015827',
 'REACT:R-HSA-1015869',
 'REACT:R-HSA-1017213',
 'REACT:R-HSA-1017219',
 'REACT:R-HSA-1018384',
 'REACT:R-HSA-1018386',
 'REACT:R-HSA-1022113',
 'REACT:R-HSA-1022114',
 'REACT:R-HSA-1022115',
 'REACT:R-HSA-1022117',
 'REACT:R-HSA-1022134',
 'REACT:R-HSA-10

In [57]:
'REACT:R-HSA-6798695' in temp

False

In [27]:
chem_g_uni.ReactID.nunique()

2640

In [62]:
# Drop any row that has empty HINO or entity ID
dis_g_uni['Hino'] = dis_g_uni.Hino.map(lambda x: np.nan if x is None else x)
chem_g_uni['Hino'] = chem_g_uni.Hino.map(lambda x: np.nan if x is None else x)

dis_g_uni = dis_g_uni.dropna(subset=['DiseaseID', 'Hino'])
chem_g_uni = chem_g_uni.dropna(subset=['ChemicalID', 'Hino'])

In [66]:
print(dis_g_uni.shape)
print(chem_g_uni.shape)

(3015, 4)
(5229, 4)


In [69]:
# Create associations files
# Output an association file for each of chem and dis
np.savetxt(r'associations_disHINO.txt', dis_g_uni[['DiseaseID', 'Hino']].values, fmt='%s')
np.savetxt(r'associations_chemHINO.txt', chem_g_uni[['ChemicalID', 'Hino']].values, fmt='%s')

# Merge the two
subprocess.call('cat associations_disHINO.txt > hinoAssociations', shell=True)
subprocess.call('cat associations_chemHINO.txt >> hinoAssociations', shell=True)

0

In [70]:
# Make entities file
entities = dis_g_uni.DiseaseID.unique().tolist() + chem_g_uni.ChemicalID.unique().tolist()
np.savetxt(r'HINOentities.lst', entities, fmt='%s')

### Running OPA

In [71]:
# Ok let's actually run it 
subprocess.check_output('(cd ../../opa2vec/ ; python2 runOPA2Vec.py -ontology ../ontologies/hino.owl -associations ../msc-thesis/opa/hinoAssociations -entities ../msc-thesis/opa/HINOentities.lst -outfile ../msc-thesis/opa/hinoVecs.lst)', shell=True)

b'Loading of Axioms ...\nLoading 1 of 6 ...\n    1%\n    2%\n    3%\n    4%\n    5%\n    6%\n    7%\n    8%\n    9%\n    10%\n    12%\n    13%\n    15%\n    17%\n    18%\n    20%\n    21%\n    23%\n    25%\n    26%\n    28%\n    29%\n    31%\n    32%\n    34%\n    36%\n    38%\n    39%\n    41%\n    43%\n    45%\n    48%\n    50%\n    53%\n    55%\n    57%\n    59%\n    61%\n    64%\n    66%\n    69%\n    71%\n    73%\n    76%\n    77%\n    79%\n    81%\n    83%\n    85%\n    87%\n    89%\n    90%\n    92%\n    94%\n    96%\n    97%\n    ... finished\nLoading 2 of 6 ...\n    25%\n    ... finished\nLoading 3 of 6 ...\n    ... finished\nLoading 4 of 6 ...\n    39%\n    ... finished\nLoading 5 of 6 ...\n    5%\n    33%\n    ... finished\nLoading 6 of 6 ...\n    ... finished\n    ... finished\nProperty Saturation Initialization ...\n    ... finished\nReflexive Property Computation ...\n    ... finished\nObject Property Hierarchy and Composition Computation ...\n    ... finished\nContext In

### Finished OPA

In [82]:
# # Use regex to get the reactome ID from Definitions
# r = re.compile('Reactome DB_ID: [0-9]*')
# vmatch = np.vectorize(lambda x:bool(r.match(x)))

# # func to extract the reactome bit
# def get_react_id(x, r):
#     if x is not None:
#         results = r.search(x)
#         if results is None: return None
#         else: return results[0].replace('Reactome DB_ID: ', 'REACT:R-HSA-')
#     else:
#         return None

# # Convert to str and extract
# hino['Definitions'] = hino.Definitions.astype(str)
# hino['Definitions'] = hino.Definitions.map(lambda x: get_react_id(x,r))

# hino = hino.dropna(subset=['Definitions'])
# print(len(hino))

# # make the map
# react_map = dict(zip(hino.Definitions, hino['Class ID']))

9119


In [83]:
hino.head()

,Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,alternative term,bearer of,...,starts,starts before,starts during,starts with,Subject and Keywords,temporal interpretation,temporally related to,term editor,Title,transitively provides input for
7,http://purl.obolibrary.org/obo/HINO_0025202,"Protein kinase C (alpha, gamma, delta)",NaN,REACT:R-HSA-112001,False,NaN,NaN,http://purl.obolibrary.org/obo/PR_000000001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,http://purl.obolibrary.org/obo/HINO_0020975,GPCR:ligand complexes that act on Gs:Heterotri...,NaN,REACT:R-HSA-744890,False,NaN,NaN,http://purl.obolibrary.org/obo/INO_0000090,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,http://purl.obolibrary.org/obo/HINO_0006390,C4A,NaN,REACT:R-HSA-166732,False,NaN,NaN,http://purl.obolibrary.org/obo/INO_0000090,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,http://purl.obolibrary.org/obo/HINO_0020108,Alpha-1(XXVII) propeptides,NaN,REACT:R-HSA-2268919,False,NaN,NaN,http://purl.obolibrary.org/obo/PR_000000001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,http://purl.obolibrary.org/obo/HINO_0013000,Diacylglycerol kinase,NaN,REACT:R-HSA-426070,False,NaN,NaN,http://purl.obolibrary.org/obo/PR_000000001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Create map of REACT ID to URI

# Make the maps from this


In [104]:
# # Now we have a react-hino map we need a uniprot-react map
# reactome = pd.read_csv('../UniProt2Reactome.txt', delimiter="\t", header=None) #error_bad_lines=False)
# reactome.rename(columns={1:'ReactID', 0:'UniprotID'}, inplace=True)
# uniprot_react = dict(zip(reactome['UniprotID'], reactome['ReactID']))

In [157]:
# read in chem-path and dis-path
# create associations file
# create entities file
# run opa2vec
# Switch to opa-nn and import the vectors

In [158]:
# # Read in Chem path
# df_cpath = pd.read_csv('../ctd-to-nt/csvs/CTD_chem_pathways_enriched.csv.gz', skiprows=27)
# df_cpath = df_cpath.drop(0)

# # Read in dis path
# df_dp = pd.read_csv('../ctd-to-nt/csvs/CTD_diseases_pathways.csv', skiprows=27)
# df_dp = df_dp.drop(0)

In [159]:
# # Create association file
# df_cpath.sample(3)

,# ChemicalName,ChemicalID,CasRN,PathwayName,PathwayID,PValue,CorrectedPValue,TargetMatchQty,TargetTotalQty,BackgroundMatchQty,BackgroundTotalQty
18193,2-(2-amino-3-methoxyphenyl)-4H-1-benzopyran-4-one,C093973,NaN,Pancreatic secretion,KEGG:hsa04972,1.330000e-09,0.000002,11.0,391.0,96.0,43067.0
201894,Benzamides,D001549,NaN,Arginine biosynthesis,KEGG:hsa00220,8.960000e-09,0.000020,13.0,4297.0,21.0,43067.0
70017,"7-(benzylamino)-1,3,4,8-tetrahydropyrrolo(4,3,...",C553817,NaN,Thyroid hormone signaling pathway,KEGG:hsa04919,1.420000e-07,0.000100,6.0,87.0,116.0,43067.0


In [160]:
# # ChemicalID PathwayID URI
# df_cpath['URI'] = df_cpath.PathwayID.map(lambda x: react_map.get(x))
# # # Apply the maps to df1
# # df1['DOID'] = df1.DiseaseID.map(lambda x: dis_map.get(x))
# # df1['CID'] = df1.ChemicalID.map(lambda x: chem_map.get(x))

In [162]:
# df_cpath.URI.nunique()

0

In [47]:
# tester = df_cpath.PathwayID.unique()
# tester = [x for x in tester if 'REACT' in x]
# sorted(tester)

In [48]:
# tester2 = react_map.keys()
# print(len(tester2))
# sorted(tester2)

In [49]:
# 'REACT:R-HSA-9033520' in tester2

In [50]:
# len(react_map)

In [ ]:
# Get uniprot ids for chem dis (already have somewhere)
# Create map of uniprot to HINO as above
# Game set match